### Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import timm
from model import LatenViTSmall
import os
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from PIL import Image


c:\Users\Haseeb\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Configuration

In [9]:

class Config:
    MODEL_NAME   = 'vit_small_patch16_224.augreg_in21k'
    NUM_CLASSES  = 7      
    NREPEAT      = 3
    START_BLOCK  = 4
    END_BLOCK    = 10
    NUM_LAYERS   = 11
    
    BATCH_SIZE   = 64
    NUM_EPOCHS   = 5
    LEARNING_RATE= 1e-4
    DEVICE       = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    DATA_ROOT    = "../../../pacs_data/pacs_data"
    DOMAINS      = ["art_painting", "cartoon", "photo", "sketch"]
    
    TRANSFORM = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))
])


### PACS Dataset Class

In [3]:
class PACSDataset(Dataset):
    def __init__(self, root_dir, domain, transform=None):
        self.root_dir    = os.path.join(root_dir, domain)
        self.transform   = transform
        self.classes     = sorted(os.listdir(self.root_dir))
        self.class_to_idx= {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.images      = []
        self.labels      = []
        
        for cls_name in self.classes:
            cls_dir = os.path.join(self.root_dir, cls_name)
            for img_name in os.listdir(cls_dir):
                self.images.append(os.path.join(cls_dir, img_name))
                self.labels.append(self.class_to_idx[cls_name])
                
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        image    = Image.open(img_path).convert('RGB')
        label    = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        return image, label


### Model Setup

In [4]:
def setup_model():
    base_model = timm.create_model(Config.MODEL_NAME, pretrained=True)
    model = LatenViTSmall(
        model     = base_model,
        nrepeat   = Config.NREPEAT,
        start     = Config.START_BLOCK,
        end       = Config.END_BLOCK,
        num_layers= Config.NUM_LAYERS
    )
    return model.to(Config.DEVICE)

def setup_baseline_model():
    base_model = timm.create_model(Config.MODEL_NAME, pretrained=True)
    base_model.head = nn.Linear(base_model.head.in_features, Config.NUM_CLASSES)
    return base_model.to(Config.DEVICE)


### Training Function

In [5]:
def train_epoch(model, dataloader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct      = 0
    total        = 0
    
    for images, labels in dataloader:
        images, labels = images.to(Config.DEVICE), labels.to(Config.DEVICE)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss    = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted  = outputs.max(1)
        total        += labels.size(0)
        correct      += predicted.eq(labels).sum().item()
        
    epoch_loss = running_loss / len(dataloader)
    epoch_acc  = 100.0 * correct / total
    return epoch_loss, epoch_acc


### Training Function

In [6]:
@torch.no_grad()
def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total   = 0
    
    for images, labels in dataloader:
        images, labels = images.to(Config.DEVICE), labels.to(Config.DEVICE)
        outputs        = model(images)
        
        _, predicted = outputs.max(1)
        total       += labels.size(0)
        correct     += predicted.eq(labels).sum().item()
        
    return 100.0 * correct / total


### Baseline -CotFormer

In [10]:

train_loaders = []
test_loaders  = []
for domain in Config.DOMAINS:
    ds_train = PACSDataset(Config.DATA_ROOT, domain, Config.TRANSFORM)
    ds_test  = PACSDataset(Config.DATA_ROOT, domain, Config.TRANSFORM)
    train_loaders.append(DataLoader(ds_train, batch_size=Config.BATCH_SIZE, shuffle=True))
    test_loaders .append(DataLoader(ds_test,  batch_size=Config.BATCH_SIZE, shuffle=False))


full_train_ds = ConcatDataset([dl.dataset for dl in train_loaders])
full_test_ds  = ConcatDataset([dl.dataset  for dl in test_loaders ])
full_train_loader = DataLoader(full_train_ds, batch_size=Config.BATCH_SIZE, shuffle=True)
full_test_loader  = DataLoader(full_test_ds,  batch_size=Config.BATCH_SIZE, shuffle=False)

model     = setup_model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=Config.LEARNING_RATE)

for epoch in range(1, Config.NUM_EPOCHS + 1):
    loss, acc = train_epoch(model, full_train_loader, criterion, optimizer)
    print(f"[Epoch {epoch}/{Config.NUM_EPOCHS}] Train Loss: {loss:.4f}, Train Acc: {acc:.2f}%")

test_acc = evaluate(model, full_test_loader)
print(f"Baseline (all domains) Test Accuracy: {test_acc:.2f}%")

from collections import OrderedDict
print("\n[Per-Domain Evaluation]")
domain_accuracies = OrderedDict()
for domain, loader in zip(Config.DOMAINS, test_loaders):
    acc = evaluate(model, loader)
    domain_accuracies[domain] = acc
    print(f"  {domain:>12}: {acc:.2f}%")

[Epoch 1/5] Train Loss: 1.4687, Train Acc: 53.45%
[Epoch 2/5] Train Loss: 0.6216, Train Acc: 76.32%
[Epoch 3/5] Train Loss: 0.3985, Train Acc: 85.04%
[Epoch 4/5] Train Loss: 0.2210, Train Acc: 91.95%
[Epoch 5/5] Train Loss: 0.1485, Train Acc: 94.92%
Baseline (all domains) Test Accuracy: 97.10%

[Per-Domain Evaluation]
  art_painting: 96.68%
       cartoon: 98.76%
         photo: 99.64%
        sketch: 95.24%


### Baseline Vanilla

In [8]:

train_loaders = []
test_loaders  = []
for domain in Config.DOMAINS:
    ds_train = PACSDataset(Config.DATA_ROOT, domain, Config.TRANSFORM)
    ds_test  = PACSDataset(Config.DATA_ROOT, domain, Config.TRANSFORM)
    train_loaders.append(DataLoader(ds_train, batch_size=Config.BATCH_SIZE, shuffle=True))
    test_loaders .append(DataLoader(ds_test,  batch_size=Config.BATCH_SIZE, shuffle=False))


full_train_ds = ConcatDataset([dl.dataset for dl in train_loaders])
full_test_ds  = ConcatDataset([dl.dataset  for dl in test_loaders ])
full_train_loader = DataLoader(full_train_ds, batch_size=Config.BATCH_SIZE, shuffle=True)
full_test_loader  = DataLoader(full_test_ds,  batch_size=Config.BATCH_SIZE, shuffle=False)

model     = setup_baseline_model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=Config.LEARNING_RATE)

for epoch in range(1, Config.NUM_EPOCHS + 1):
    loss, acc = train_epoch(model, full_train_loader, criterion, optimizer)
    print(f"[Epoch {epoch}/{Config.NUM_EPOCHS}] Train Loss: {loss:.4f}, Train Acc: {acc:.2f}%")

test_acc = evaluate(model, full_test_loader)
print(f"Baseline (all domains) Test Accuracy: {test_acc:.2f}%")

print("\n[Per-Domain Evaluation]")
domain_accuracies = OrderedDict()
for domain, loader in zip(Config.DOMAINS, test_loaders):
    acc = evaluate(model, loader)
    domain_accuracies[domain] = acc
    print(f"  {domain:>12}: {acc:.2f}%")

[Epoch 1/5] Train Loss: 0.2591, Train Acc: 91.09%
[Epoch 2/5] Train Loss: 0.0579, Train Acc: 98.12%
[Epoch 3/5] Train Loss: 0.0398, Train Acc: 98.93%
[Epoch 4/5] Train Loss: 0.0411, Train Acc: 98.73%
[Epoch 5/5] Train Loss: 0.0213, Train Acc: 99.23%
Baseline (all domains) Test Accuracy: 98.92%

[Per-Domain Evaluation]
  art_painting: 99.61%
       cartoon: 99.70%
         photo: 99.34%
        sketch: 97.91%
